In [ ]:
import pandas as pd
df = pd.read_csv("/content/heart (1).csv")

df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# 1. Load dataset
df = pd.read_csv("/content/heart (1).csv")

# 2. Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns

# 3. Apply Label Encoding and One Hot Encoding
df_encoded = df.copy()
label_encoders = {}

for col in categorical_cols:
    if df[col].nunique() == 2:
        # Binary column: Label encode
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    else:
        # Multi-category column: One-hot encode
        df_encoded = pd.get_dummies(df_encoded, columns=[col])

# 4. Separate features and labels
X = df_encoded.drop("HeartDisease", axis=1)  # assuming target column is named 'target'
y = df_encoded["HeartDisease"]

# 5. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6. Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 7. Define models
models = {
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier()
}

# 8. Train and evaluate each model
print("=== Accuracy without PCA ===")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name}: {acc:.4f}")

# 9. Apply PCA
pca = PCA(n_components=0.95)  # retain 95% variance
X_pca = pca.fit_transform(X_scaled)

# Split PCA-transformed data
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# 10. Retrain and evaluate models with PCA
print("\n=== Accuracy with PCA ===")
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    y_pred = model.predict(X_test_pca)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} (with PCA): {acc:.4f}")


=== Accuracy without PCA ===
SVM: 0.8804
Logistic Regression: 0.8533
Random Forest: 0.8913

=== Accuracy with PCA ===
SVM (with PCA): 0.8804
Logistic Regression (with PCA): 0.8533
Random Forest (with PCA): 0.8587
